In [27]:
import pandas
import numpy
import random
import sys

from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.layers import LSTM, GRU
from keras.optimizers import RMSprop, Adam

In [2]:
cd ..

/Users/mmccurdy/code/starwars-namer


In [3]:
from swnamer.process import chunk_names, chunk_names_file, create_indices_file, create_indices, SampleNames, SampleNamesFile, create_training_vectors

In [4]:
male = pandas.read_csv('data/male.txt', header=5, names=['name'])

In [5]:
female = pandas.read_csv('data/female.txt', header=5, names=['name'])

In [6]:
male.shape, female.shape

((2942, 1), (5000, 1))

In [7]:
male.columns

Index(['name'], dtype='object')

In [8]:
male.sample(20)

,name
2139,Ricky
1143,Henrique
2840,Wilson
2826,Wilhelm
2174,Rodrick
201,Averell
620,Derrin
441,Chauncey
11,Abel
1439,Kendrick


In [9]:
female.sample(20)

,name
1553,Ellissa
1115,Corrianne
2230,Inez
795,Caroljean
2968,Leta
1838,Frankie
1234,Darya
1538,Elissa
867,Celeste
2179,Holly-Anne


In [10]:
combined = pandas.concat([male, female])

In [11]:
combined = combined.sample(frac=1, random_state=38974)

In [12]:
combined

,name
2896,Laurianne
721,Elbert
199,Austin
301,Anni
645,Dionis
2475,Tab
2211,Rubin
4037,Randene
728,Eliot
773,Carmelina


In [13]:
combined.loc[:, 'name'] = combined.name.str.lower()

In [14]:
combined

,name
2896,laurianne
721,elbert
199,austin
301,anni
645,dionis
2475,tab
2211,rubin
4037,randene
728,eliot
773,carmelina


In [15]:
timesteps = 1

In [16]:
token_to_index, index_to_token = create_indices(combined, 'name')

In [ ]:
chunks, next_char = chunk_names(combined, 'name', timesteps)

In [ ]:
vocab_size = len(token_to_index)

In [ ]:
X, y = create_training_vectors(chunks, next_char, token_to_index, timesteps, vocab_size)

In [ ]:
X.shape

In [ ]:
y.shape

In [17]:
model = Sequential()
model.add(LSTM(128, input_shape=(timesteps, vocab_size)))
model.add(Dense(vocab_size))
model.add(Activation('softmax'))
optimizer = Adam(lr=.01, clipvalue=5)
model.compile(optimizer, 'categorical_crossentropy')

In [ ]:
model.fit(X, y, epochs=200, batch_size=128, callbacks=[SampleNames(chunks, timesteps, vocab_size, token_to_index, index_to_token)])

In [ ]:
combined.to_csv('output/standard_names.csv', index=False, header=False)

In [4]:
with open('output/standard_names.csv', 'r') as infile:
    text = infile.read()

In [5]:
token_to_index, index_to_token = create_indices_file(text)

In [6]:
vocab_size = len(token_to_index)

In [48]:
timesteps = 2

In [49]:
chunks, next_char = chunk_names_file(text, timesteps, stepsize=1)

In [50]:
X, y = create_training_vectors(chunks, next_char, token_to_index, timesteps, vocab_size)

In [56]:
model = Sequential()
model.add(LSTM(64, input_shape=(timesteps, vocab_size), return_sequences=True))
model.add(LSTM(64, input_shape=(timesteps, vocab_size), return_sequences=True))
model.add(LSTM(64, input_shape=(timesteps, vocab_size), return_sequences=False))

model.add(Dense(vocab_size))
model.add(Activation('softmax'))
optimizer = RMSprop(lr=.01, clipvalue=1)
model.compile(optimizer, 'categorical_crossentropy')

In [57]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_17 (LSTM)               (None, 2, 64)             24320     
_________________________________________________________________
lstm_18 (LSTM)               (None, 2, 64)             33024     
_________________________________________________________________
lstm_19 (LSTM)               (None, 64)                33024     
_________________________________________________________________
dense_15 (Dense)             (None, 30)                1950      
_________________________________________________________________
activation_15 (Activation)   (None, 30)                0         
Total params: 92,318
Trainable params: 92,318
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(X, y, epochs=200, batch_size=64, callbacks=[SampleNamesFile(timesteps, vocab_size, token_to_index, index_to_token, text)])

Epoch 1/200
55853/55853 [==============================] - 10s 184us/step - loss: 2.4132

----- Generating text after Epoch: 0
----- diversity: 0.2
----- Generating with seed: "w"
wen
----- diversity: 0.5
----- Generating with seed: "w"
wel
----- diversity: 1.0
----- Generating with seed: "w"
wpinart
----- diversity: 1.2
----- Generating with seed: "w"
wolie
Epoch 2/200
55853/55853 [==============================] - 7s 128us/step - loss: 2.2645
Epoch 3/200
55853/55853 [==============================] - 6s 104us/step - loss: 2.2247
Epoch 4/200
55853/55853 [==============================] - 6s 100us/step - loss: 2.2074
Epoch 5/200
55853/55853 [==============================] - 6s 111us/step - loss: 2.1958
Epoch 6/200
55853/55853 [==============================] - 7s 133us/step - loss: 2.1878

----- Generating text after Epoch: 5
----- diversity: 0.2
----- Generating with seed: "u"
unda
----- diversity: 0.5
----- Generating with seed: "u"
un
----- diversity: 1.0
----- Generating with seed

/Users/mmccurdy/code/starwars-namer/swnamer/process.py:93: RuntimeWarning: divide by zero encountered in log
  preds = numpy.log(preds) / temperature


55853/55853 [==============================] - 6s 111us/step - loss: 2.1827
Epoch 38/200
55853/55853 [==============================] - 6s 115us/step - loss: 2.1858
Epoch 39/200
55853/55853 [==============================] - 6s 107us/step - loss: 2.1889
Epoch 40/200
55853/55853 [==============================] - 6s 109us/step - loss: 2.1910
Epoch 41/200
55853/55853 [==============================] - 6s 107us/step - loss: 2.1911

----- Generating text after Epoch: 40
----- diversity: 0.2
----- Generating with seed: "u"
uin
----- diversity: 0.5
----- Generating with seed: "u"
uenne
----- diversity: 1.0
----- Generating with seed: "u"
ulynn
----- diversity: 1.2
----- Generating with seed: "u"
uas
Epoch 42/200
55853/55853 [==============================] - 6s 112us/step - loss: 2.1944
Epoch 43/200
55853/55853 [==============================] - 6s 105us/step - loss: 2.1935
Epoch 44/200
55853/55853 [==============================] - 6s 104us/step - loss: 2.1974
Epoch 45/200
55853/55853 [====